In [1]:
import pandas as pd
import numpy as np
WDIR = './data/'

In [2]:
business_json_path = WDIR + "yelp_academic_dataset_business.json"
checkin_json_path = WDIR + "yelp_academic_dataset_checkin.json"
tip_json_path = WDIR + "yelp_academic_dataset_tip.json"
reviews_json_path = WDIR + "yelp_academic_dataset_review.json"

In [3]:
# Read in businesses df
df_b = pd.read_json(business_json_path, lines=True)

In [5]:
# Maintain a relevant set of businesses
relevant_businesses = set(df_b[df_b['state'].isin(['VA', 'NC'])].business_id.unique())

# Utility function to read in large dataframes and filter on the fly
def filter(path, critera, size=1000):
    chunk_list = []
    reader = pd.read_json(path, lines=True, chunksize=size)
    
    for chunk in reader:
        chunk_filter = chunk[chunk.business_id.isin(critera)]
        chunk_list.append(chunk_filter)
    
    reader.close()
    return pd.concat(chunk_list)

In [ ]:
# Read and save reviews file
df_revs = filter(reviews_json_path, relevant_businesses, 1000)
df_concat.to_pickle(WDIR + 'dfs/reviews.pkl')

In [68]:
# Read and save checkin file
df_checkin = filter(checkin_json_path, relevant_businesses, 1000)
df_checkin.to_pickle(WDIR + 'dfs/checkin.pkl')

In [ ]:
# Read and save tip file
df_tips = filter(tip_json_path, relevant_businesses, 1000)
df_tips.to_pickle(WDIR + 'dfs/tips.pkl')

In [5]:
import pickle
with open('reviews.pkl', 'rb') as pickle_file:
    df_revs = pickle.load(pickle_file)
    
with open('checkin.pkl', 'rb') as pickle_file:
    df_checkin = pickle.load(pickle_file)
    
with open('tips.pkl', 'rb') as pickle_file:
    df_tips = pickle.load(pickle_file)

In [22]:
bus_rev_dict = {}
for x in relevant_businesses:
    bus_rev_dict[x] = []
for x,c in df_revs.iterrows():
    try:
        bus_rev_dict[c['business_id']].append(c['text'])
    except:
        pass

In [23]:
for x,c in df_revs.iterrows():
    try:
        bus_rev_dict[c['business_id']].append(c['text'])
    except:
        pass

In [44]:
from textblob import TextBlob
import nltk

In [45]:
def findavgsentiment(textarr):
    totalSent = 0
    for text in textarr:
        blob = TextBlob(text)
        totalSent += blob.sentiment.polarity
    return totalSent/len(textarr)

In [52]:
bus_score_dict = {}
for x in relevant_businesses:
    bus_score_dict[x] = []
for item in bus_rev_dict.items():
    bus_score_dict[item[0]] = findavgsentiment(item[1])
    

In [53]:
bus_score_dict

{'fJkXfvCiVGJsjrIrUuLJCQ': 0.26234487508453674,
 'FqVTaF56MSgqCCX3N6hgLQ': 0.37071906565656565,
 'sV6kKpzneWwQ8cY8gQ8bgA': 0.263858991033446,
 'R2ir667KUOdyhGJp9Q06dQ': 0.24828639331498806,
 '4GrFgFdt3fYdZsYbnyRlZg': 0.10212637741046832,
 'x5fgrhaq8R8qXhZFPnfXWw': 0.2849578781257856,
 'VKdVPVkMOotZ-J248lHReQ': 0.25297619047619047,
 'pCb0jYxksLR562vV7As1bQ': 0.2867355784277791,
 'GlQs6zeCN0dwCe7gMBXFsg': 0.3225956664862915,
 'jdft5jzXyceJZLRtVL38VA': 0.13932570912903125,
 'iqx8VckOhO7nict32SPjKg': 0.13101826306357292,
 'wUHPgIh-rJThxPvxHPlsjQ': 0.1637833522721307,
 '4y8LLc9tuxqKhDSeRDUTDA': 0.2587474829916635,
 'huTkwb90PQw4HqGkbhz74Q': 0.2993022486772487,
 'VXFycyyI3uEx6IOHkPLvSQ': 0.3499585578832374,
 'vi2UlcBHPlGMTbGWO1e2-w': 0.3531944444444444,
 'l9tFVg7r7QXuLnCywMQl7Q': 0.29586083920224054,
 'ic5TJB0pUeiQ53J2CRBMig': 0.15404577234934375,
 'esB5D7EbZ5mJkqwSBK1yOA': -0.00465277777777779,
 '1Dx2W-2GL9LaE9j2vD5lfw': 0.13834928075637923,
 'at63ej-ri33Vqx4333Rd3Q': 0.12056925047861677,
 

In [ ]:
# Aggregate checkins (still have to merge)

In [86]:
df_checkin['date'] = df_checkin['date'].apply(lambda x : list(map(lambda y : datetime.strptime(y, "%Y-%m-%d %H:%M:%S"), x.split(", "))))

In [106]:
df_checkin['checkins'] = df_checkin['date'].apply(len)

In [129]:
df_checkin['interval'] = df_checkin['date'].apply(lambda x : (x[-1] - x[0]).days if (x[-1] != x[0]) else 0)

In [119]:
df_checkin

,business_id,date,checkins,interval
2,--7zmmkVg-IMGaXbuVd0SQ,"[2014-12-29 19:25:50, 2015-01-17 01:49:14, 201...",193,1805
9,--EX4rRznJrltyn-34Jz1w,"[2010-02-26 17:05:40, 2012-12-29 20:05:04, 201...",17,3289
15,--KCl2FvVQpvjzmZSPyviA,"[2011-07-29 16:53:35, 2011-08-25 00:19:18, 201...",107,3041
22,--U98MNlDym2cLn36BBPgQ,"[2011-10-05 22:50:41, 2012-04-11 00:06:36, 201...",3,286
28,--cZ6Hhc9F7VkKXxHMVZSQ,"[2010-04-24 19:48:58, 2010-05-01 19:28:23, 201...",753,3517
34,--irMpXK9y_xLlXPvPAQvw,"[2011-05-05 12:20:54, 2012-01-25 21:09:52, 201...",17,2935
68,-0QtTRrAMn6DKLZNef3Ojg,"[2016-07-23 20:51:17, 2017-04-04 18:13:06, 201...",3,941
88,-0lbeUxS9QDBtIS5KtPKZw,[2018-08-16 13:21:18],1,0
98,-0y3MZU2oYP8r1ruDP1bfQ,"[2015-04-11 13:14:14, 2015-11-21 16:05:56, 201...",5,924
147,-2HjuT4yjLZ3b5f_abD87Q,"[2010-07-23 10:45:22, 2010-07-23 10:49:35, 201...",6,3367


In [5]:
# filter business 
df_b = df_b[df_b['state'].isin(['VA', 'NC'])]

In [43]:
# convert categories label to iterables and then encode
a = df_b['categories'].apply(lambda x : str(x).split(', '))

In [44]:
# Use this to encode categories
from sklearn.preprocessing import MultiLabelBinarizer
b = MultiLabelBinarizer()
b.fit_transform(a.values)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
# @Todo:
"""

--> EDA:
    random scatter plots / histograms


-- > Preprocessing
- Merge checkins df (checkins, and interval) on business id with bussinesses df (filtered)
- Add sentiment column to business 
- Encode categories column in business dataframe (might want to filter categories to top N to reduce dimensionality)
- Encode 'attributes column' in business dataframe 
- Vectorize reviews column (do not have to do for this checkpoint)

---> SMOTE data before running any models (accounts for classification with an imbalanced class distr.)

--> run some basic models and note down the metrics
"""